In [ ]:
import os
import glob
import json
import numpy as np
import pandas as pd
import statistics
from pathlib import Path 
from Bio.PDB import PDBParser
from scipy.spatial import distance


def create_structure_interaction_map(pdb_file, distance_cutoff=8):
    """
    Create an interaction map for a protein structure from a PDB file.

    Parameters:
    pdb_file (str): Path to the PDB file.
    distance_cutoff (float): Distance cutoff for interactions. Default is 8.

    Returns:
    interaction_map (np.array): Interaction map of the protein structure.
    protein_a_len (int): Length of protein A.
    protein_b_len (int): Length of protein B.
    """
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('PDB', pdb_file)
    chains = list(structure.get_chains())
    
    if len(chains) < 2:
        raise ValueError("Structure must contain at least two chains.")
    
    coords_A, coords_B = [], []
    for index, chain in enumerate(chains[:2]):
        for residue in chain:
            if 'CB' in residue or 'CA' in residue:
                atom_name = 'CB' if 'CB' in residue else 'CA'
                coords = residue[atom_name].get_coord()
                if index == 0:
                    coords_A.append(coords)
                else:
                    coords_B.append(coords)
    
    distances = distance.cdist(coords_A, coords_B)
    interaction_map = (distances <= distance_cutoff).astype(int)
    
    protein_a_len = sum(1 for res in structure.get_residues() if res.get_parent().id == 'A')
    protein_b_len = sum(1 for res in structure.get_residues() if res.get_parent().id == 'B')

    return interaction_map, protein_a_len, protein_b_len

def get_pLDDT_scores(pdb_file):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('PDB', pdb_file)
    
    # Use dictionary comprehension to directly create pLDDT_scores
    pLDDT_scores = {(chain.id, i+1): residue['CA'].get_bfactor() 
                    for model in structure 
                    for chain in model 
                    for i, residue in enumerate(chain)}
    
    return pLDDT_scores

def calculate_pLDDT_scores(interaction_map1, interaction_map2, pdb_file):
    parser = PDBParser()
    structure = parser.get_structure('PDB', pdb_file)
    
    # total_bfactors = []
    chain_bfactors = {}  # This dictionary will store the average B-factors for each chain

    for model in structure:
        for chain in model:
            bfactors = [atom.get_bfactor() for atom in chain.get_atoms() if atom.get_bfactor() is not None]
            if bfactors:
                chain_bfactors[chain.id] = np.mean(bfactors)
                # total_bfactors.extend(bfactors)

    # Retrieve the values for chain 'A' and 'B', returning None if they are not found
    whole_pLDDT_A = chain_bfactors.get('A', None)
    whole_pLDDT_B = chain_bfactors.get('B', None)
                
    pLDDT_scores = get_pLDDT_scores(pdb_file)
    interaction_maps = [interaction_map1, interaction_map2]
    results = []
    for interaction_map in interaction_maps:
        if np.count_nonzero(interaction_map) == 0:
            results.append((0, 0, 0))
            continue

        interaction_indices = np.where(interaction_map != 0)
        positions_A = interaction_indices[0]
        positions_B = interaction_indices[1]

        unique_positions_A = np.unique(positions_A)
        unique_positions_B = np.unique(positions_B)

        # Initialize default values
        pLDDT_A = []
        average_pLDDT_A = 0
        if unique_positions_A.size > 0:
            pLDDT_A = [pLDDT_scores.get(('A', pos), 0) for pos in unique_positions_A]
            average_pLDDT_A = np.mean(pLDDT_A) if pLDDT_A else 0

        pLDDT_B = []
        average_pLDDT_B = 0
        if unique_positions_B.size > 0:
            pLDDT_B = [pLDDT_scores.get(('B', pos), 0) for pos in unique_positions_B]
            average_pLDDT_B = np.mean(pLDDT_B) if pLDDT_B else 0

        # Calculate average pLDDT score in the interface
        pLDDT_interface = pLDDT_A + pLDDT_B
        average_pLDDT_interface = np.mean(pLDDT_interface) if pLDDT_interface else 0

        results.append((average_pLDDT_A, average_pLDDT_B, average_pLDDT_interface))

    return results, whole_pLDDT_A, whole_pLDDT_B

def load_pae_data(pae_file):
    try:
        with open(pae_file, 'r') as file:
            json_data = json.load(file)

        # Check if all required keys are present
        required_keys = {"plddt", "ptm", "iptm", "pae"}
        if not required_keys <= json_data.keys():
            print(f"Missing data in PAE file {pae_file}.")
            return None

        plddt = statistics.mean(json_data["plddt"])
        ptm = json_data["ptm"]
        iptm = json_data["iptm"]
        pae = np.array(json_data['pae'])
        return plddt, ptm, iptm, pae
    except Exception as e:
        print(f"Error loading PAE data from {pae_file}: {e}")
        return None


def calculate_interaction_areas(pae, protein_a_len, pae_cutoff, interaction_map):
    thresholded_pae = np.where(pae < pae_cutoff, 1, 0)
    
    # Extract interaction areas for AB and BA
    area_ab = thresholded_pae[:protein_a_len, protein_a_len:]
    area_ba = thresholded_pae[protein_a_len:, :protein_a_len]
    
    # Pad interaction_map to match the shape of area_ab if necessary
    pad_rows_im = area_ab.shape[0] - interaction_map.shape[0]
    pad_cols_im = area_ab.shape[1] - interaction_map.shape[1]
    padded_interaction_map = np.pad(interaction_map, ((0, pad_rows_im), (0, pad_cols_im)), mode='constant', constant_values=0)
    
    # Pad transpose of interaction_map to match the shape of area_ba if necessary
    pad_rows_imt = area_ba.shape[0] - interaction_map.T.shape[0]
    pad_cols_imt = area_ba.shape[1] - interaction_map.T.shape[1]
    padded_interaction_map_t = np.pad(interaction_map.T, ((0, pad_rows_imt), (0, pad_cols_imt)), mode='constant', constant_values=0)
    
    # Calculate contacts where both PAE and interaction_map indicate interaction
    contact_ab = np.where((area_ab > 0) & (padded_interaction_map > 0), 1, 0)
    contact_ba = np.where((area_ba > 0) & (padded_interaction_map_t > 0), 1, 0)
    
    return area_ab, area_ba, contact_ab, contact_ba


def comparing_interaction_area(area_ab, area_ba, contact_ab, contact_ba):
    local_interaction_area_ab = np.sum(area_ab)
    local_interaction_area_ba = np.sum(area_ba)
    local_interaction_area_both = local_interaction_area_ab + local_interaction_area_ba
    contact_local_interaction_area_ab = np.sum(contact_ab)
    contact_local_interaction_area_ba = np.sum(contact_ba)
    contact_local_interaction_area = contact_local_interaction_area_ab + contact_local_interaction_area_ba
    return local_interaction_area_ab, local_interaction_area_ba, local_interaction_area_both, contact_local_interaction_area_ab, contact_local_interaction_area_ba, contact_local_interaction_area

def calculate_pae_values(pae, area_ab, area_ba, contact_ab, contact_ba, protein_a_len, pae_cutoff):
    domain_pae_ab = pae[:protein_a_len, protein_a_len:][area_ab==1]
    domain_pae_ba = pae[protein_a_len:, :protein_a_len][area_ba==1]
    contact_pae_ab = pae[:protein_a_len, protein_a_len:][contact_ab==1]
    contact_pae_ba = pae[protein_a_len:, :protein_a_len][contact_ba==1]
    mean_domain_pae_ab = np.mean(domain_pae_ab) if domain_pae_ab.size > 0 else pae_cutoff
    mean_domain_pae_ba = np.mean(domain_pae_ba) if domain_pae_ba.size > 0 else pae_cutoff
    mean_contact_pae_ab = np.mean(contact_pae_ab) if contact_pae_ab.size > 0 else pae_cutoff 
    mean_contact_pae_ba = np.mean(contact_pae_ba) if contact_pae_ba.size > 0 else pae_cutoff
    return mean_domain_pae_ab, mean_domain_pae_ba, mean_contact_pae_ab, mean_contact_pae_ba

def calculate_inversed_pae(mean_domain_pae_ab, mean_domain_pae_ba, mean_contact_pae_ab, mean_contact_pae_ba, pae_cutoff):
    inversed_mean_domain_pae_ab = 1 - mean_domain_pae_ab / pae_cutoff
    inversed_mean_domain_pae_ba = 1 - mean_domain_pae_ba / pae_cutoff
    inversed_mean_domain_pae_both = (inversed_mean_domain_pae_ab + inversed_mean_domain_pae_ba)/2
    inversed_mean_contact_pae_ab = 1 - mean_contact_pae_ab / pae_cutoff
    inversed_mean_contact_pae_ba = 1 - mean_contact_pae_ba / pae_cutoff
    inversed_mean_contact_pae_both = (inversed_mean_contact_pae_ab + inversed_mean_contact_pae_ba) / 2
    return inversed_mean_domain_pae_ab, inversed_mean_domain_pae_ba, inversed_mean_domain_pae_both, inversed_mean_contact_pae_ab, inversed_mean_contact_pae_ba, inversed_mean_contact_pae_both

def calculate_residue_indices(area_ab, area_ba, contact_ab, contact_ba):
    area_sum = area_ab + area_ba.transpose()
    contact_sum = contact_ab + contact_ba.transpose()
    area_residue_indices = np.nonzero(area_sum)
    contact_residue_indices = np.nonzero(contact_sum)
    return area_residue_indices, contact_residue_indices, contact_sum, area_sum

def calculate_residue_counts(area_residue_indices, contact_residue_indices):
    unique_area_residue_indices_A = np.unique(area_residue_indices[0]) + 1 # fixed on 2024/10/24
    unique_area_residue_indices_B = np.unique(area_residue_indices[1]) + 1
    area_residue_A = len(unique_area_residue_indices_A)
    area_residue_B = len(unique_area_residue_indices_B)
    area_residue_both = area_residue_A + area_residue_B

    unique_contact_residue_indices_A = np.unique(contact_residue_indices[0]) + 1
    unique_contact_residue_indices_B = np.unique(contact_residue_indices[1]) + 1
    contact_residue_A = len(unique_contact_residue_indices_A)
    contact_residue_B = len(unique_contact_residue_indices_B)
    contact_residue_both = contact_residue_A + contact_residue_B

    return area_residue_A, area_residue_B, area_residue_both, contact_residue_A, contact_residue_B, contact_residue_both, unique_area_residue_indices_A, unique_area_residue_indices_B,unique_contact_residue_indices_A, unique_contact_residue_indices_B

def process_interaction_results(pdb_file, pae_file, pae_cutoff=12, distance_cutoff=8, print_results=False):
    """
    Process the interaction results.

    Parameters:
    pdb_file (str): Path to the PDB file.
    pae_file (str): Path to the PAE file.
    pae_cutoff (float): PAE cutoff value.
    distance_cutoff (float): Distance cutoff for interactions.
    print_results (bool): Flag to indicate whether to print the results.
    
    Returns:
    interaction_data (pd.Series): Series containing the interaction results.
    """
    ## 2024/05/25 added ##
    # Check if both PDB and PAE files are available
    if not os.path.isfile(pdb_file) or not os.path.isfile(pae_file):
        print(f"Warning: One or both files are missing: PDB ({pdb_file}), PAE ({pae_file}). Skipping this analysis.")
        return None
    ## 2024/05/25 added ##
    
    # Extract protein names and rank from PDB file name
    file_name = os.path.basename(pdb_file)
    if file_name.count("___") == 1:
        protein_1 = file_name.split("___")[0]
        protein_2_temp = file_name.split("___")[1]
        protein_2 = protein_2_temp.split("_unrelaxed")[0]
    else:
        print(f"Warning: Unexpected file naming convention for {file_name}. Skipping this file.")
        return None  # or some other default behavior

    # Extract rank information
    if "_unrelaxed_rank_00" in protein_2_temp:
        rank_temp = protein_2_temp.split("_unrelaxed_rank_00")[1]
        rank = rank_temp.split("_alphafold2")[0]
    else:
        print(f"Warning: Unexpected file naming convention for {file_name}. Skipping this file.")
        return None  # or some other default behavior    
    
    # Load PAE data
    plddt, ptm, iptm, pae = load_pae_data(pae_file)
    pdb_base_folder = os.path.basename(os.path.dirname(pdb_file))
    pdb_file_name = os.path.basename(pdb_file)
    pae_file_name = os.path.basename(pae_file)

    if pae is not None:
        interaction_map, protein_a_len, protein_b_len = create_structure_interaction_map(pdb_file, distance_cutoff)
        area_ab, area_ba, contact_ab, contact_ba = calculate_interaction_areas(pae, protein_a_len, pae_cutoff, interaction_map)
        local_interaction_area_ab, local_interaction_area_ba, local_interaction_area_both, contact_local_interaction_area_ab, contact_local_interaction_area_ba, contact_local_interaction_area = comparing_interaction_area(area_ab, area_ba, contact_ab, contact_ba)
        mean_domain_pae_ab, mean_domain_pae_ba, mean_contact_pae_ab, mean_contact_pae_ba = calculate_pae_values(pae, area_ab, area_ba, contact_ab, contact_ba, protein_a_len, pae_cutoff)
        inversed_mean_domain_pae_ab, inversed_mean_domain_pae_ba, inversed_mean_domain_pae_both, inversed_mean_contact_pae_ab, inversed_mean_contact_pae_ba, inversed_mean_contact_pae_both = calculate_inversed_pae(mean_domain_pae_ab, mean_domain_pae_ba, mean_contact_pae_ab, mean_contact_pae_ba, pae_cutoff)
        area_residue_indices, contact_residue_indices, contact_sum, area_sum = calculate_residue_indices(area_ab, area_ba, contact_ab, contact_ba)
        area_residue_A, area_residue_B, area_residue_both, contact_residue_A, contact_residue_B, contact_residue_both, unique_area_residue_indices_A, unique_area_residue_indices_B,unique_contact_residue_indices_A, unique_contact_residue_indices_B = calculate_residue_counts(area_residue_indices, contact_residue_indices)
        results, pLDDT_A, pLDDT_B = calculate_pLDDT_scores(contact_sum, area_sum, pdb_file)
        contact_pLDDT_A, contact_pLDDT_B, contact_pLDDT_interface = results[0]
        domain_pLDDT_A, domain_pLDDT_B, domain_pLDDT_interface = results[1]   

        # Create a series containing the interaction results
        interaction_data = pd.Series({
            'Protein_1': protein_1,
            'Protein_2': protein_2,
            'Rank': rank,
            'integrated Local Interaction Score (Interface)': np.sqrt(inversed_mean_domain_pae_both*inversed_mean_contact_pae_both),
            'Local Interaction Score (AB)': inversed_mean_domain_pae_ab,
            'Local Interaction Score (BA)': inversed_mean_domain_pae_ba,
            'Local Interaction Score (Interface)': inversed_mean_domain_pae_both,
            'Local Interaction Area (AB)': local_interaction_area_ab,
            'Local Interaction Area (BA)': local_interaction_area_ba,
            'Local Interaction Area (Interface)': local_interaction_area_both,
            'Local Interaction Residue (A)': area_residue_A,
            'Local Interaction Residue (B)': area_residue_B,
            'Local Interaction Residue (Sum)': area_residue_both,
            'Local Interaction pLDDT (A)': domain_pLDDT_A,
            'Local Interaction pLDDT (B)': domain_pLDDT_B,
            'Local Interaction pLDDT (Interface)': domain_pLDDT_interface,
            'Contact Local Interaction Score (AB)': inversed_mean_contact_pae_ab,
            'Contact Local Interaction Score (BA)': inversed_mean_contact_pae_ba,
            'Contact Local Interaction Score (Interface)': inversed_mean_contact_pae_both,
            'Contact Local Interaction Area (AB)': contact_local_interaction_area_ab,
            'Contact Local Interaction Area (BA)': contact_local_interaction_area_ba,
            'Contact Local Interaction Area (Interface)': contact_local_interaction_area,
            'Contact Local Interaction Residue (A)': contact_residue_A,
            'Contact Local Interaction Residue (B)': contact_residue_B,
            'Contact Local Interaction Residue (Sum)': contact_residue_both,
            'Contact Local Interaction pLDDT (A)': contact_pLDDT_A,
            'Contact Local Interaction pLDDT (B)': contact_pLDDT_B,
            'Contact Local Interaction pLDDT (Interface)': contact_pLDDT_interface,
            'Local Interaction Residue Indice A': unique_area_residue_indices_A,
            'Local Interaction Residue Indice B': unique_area_residue_indices_B,
            'Contact Local Interaction Residue Indice A': unique_contact_residue_indices_A,
            'Contact Local Interaction Residue Indice B': unique_contact_residue_indices_B,
            'pLDDT': plddt,
            'pTM': ptm,
            'ipTM': iptm,
            'confidence': 0.8*iptm + 0.2*ptm,
            'Protein A Length': int(protein_a_len),
            'Protein B Length': int(protein_b_len),
            'pLDDT_A': pLDDT_A,
            'pLDDT_B': pLDDT_B,
            'pdb_file': pdb_file_name,
            'pae_json': pae_file_name,
            'pae_plot': f'{pdb_base_folder}+{protein_1}___{protein_2}_pae.png',

        })

        if print_results:
            print(interaction_data)

    return interaction_data, contact_sum, area_sum


def get_subdirectories(base_path):
    return [d.name for d in os.scandir(base_path) if d.is_dir()]

def pdb_file_info_extract(folder_path, pdb_file):
    pdb_file = os.path.join(folder_path, pdb_file)
    pae_file = pdb_file.replace(".pdb", ".json").replace("unrelaxed", "scores")
    
    file_name = os.path.basename(pdb_file)
    if file_name.count("___") == 1:
        protein_1 = file_name.split("___")[0]
        protein_2_temp = file_name.split("___")[1]
        protein_2 = protein_2_temp.split("_unrelaxed")[0]
    else:
        print(f"Warning: Unexpected file naming convention for {file_name}. Skipping this file.")
        return None  # or some other default behavior

    # Extract rank information
    if "_unrelaxed_rank_00" in protein_2_temp:
        rank_temp = protein_2_temp.split("_unrelaxed_rank_00")[1]
        rank = rank_temp.split("_alphafold2")[0]
    else:
        rank = "Not Available"  # or any default value you prefer

    return pdb_file, pae_file, protein_1, protein_2, rank


In [ ]:
from concurrent.futures import ProcessPoolExecutor, as_completed

def process_interaction_results_if_needed(pdb_file, pae_file, output_path, pae_cutoff=12, distance_cutoff=8, print_results=False):

        ## 2024/05/25 added ##
    # Check if both PDB and PAE files are available
    if not os.path.isfile(pdb_file) or not os.path.isfile(pae_file):
        print(f"Warning: One or both files are missing: PDB ({pdb_file}), PAE ({pae_file}). Skipping this analysis.")
        return None
    ## 2024/05/25 added ##
    
    file_name = os.path.basename(pdb_file)
    if file_name.count("___") == 1:
        protein_1 = file_name.split("___")[0]
        protein_2_temp = file_name.split("___")[1]
        protein_2 = protein_2_temp.split("_unrelaxed")[0]
    else:
        print(f"Warning: Unexpected file naming convention for {file_name}. Skipping this file.")
        return None  # or some other default behavior

    # Extract rank information
    if "_unrelaxed_rank_00" in protein_2_temp:
        rank_temp = protein_2_temp.split("_unrelaxed_rank_00")[1]
        rank = int(rank_temp.split("_alphafold2")[0])
    else:
        rank = "Not Available"  # or any default value you prefer
        

    output_file = os.path.join(output_path, f"{protein_1}___{protein_2}_rank_00{rank}_lis.tsv")
    
    if os.path.exists(output_file):
#         print(f"Skipping analysis for {file_name} as output already exists.")
        return None  # Return None if file exists

    interaction_data, contact_sum, area_sum = process_interaction_results(pdb_file, pae_file, pae_cutoff, distance_cutoff, print_results)
    interaction_data_df = interaction_data.to_frame().T
    interaction_data_df.to_csv(output_file, sep='\t', index=False)

    
#     interaction_data.to_csv(output_file, sep='\t', index=False)
#     interaction_data_transposed = interaction_data.transpose()
#     interaction_data_transposed.to_csv(output_file, sep='\t', index=False)


    return #f"Processed {file_name}"

def process_files_in_folder(folder, base_path, output_base, pae_cutoff, distance_cutoff):
    output_path = Path(output_base).joinpath(folder)
    output_path.mkdir(parents=True, exist_ok=True)
    
    folder_path = Path(base_path).joinpath(folder)
    
    # List comprehension to filter only files that contain '_rank_' in the filename
    pdb_files = [pdb_file for pdb_file in folder_path.rglob("*.pdb") if '_rank_00' in str(pdb_file)]

    with ProcessPoolExecutor() as executor:
        futures = [executor.submit(process_interaction_results_if_needed, str(pdb_file), str(pdb_file).replace(".pdb", ".json").replace("unrelaxed", "scores"), str(output_path), pae_cutoff, distance_cutoff) for pdb_file in pdb_files]
        for future in as_completed(futures):
            result = future.result()
            if result:
                print(result)

                
## 2024/05/30 update
def combine_tsv_files_incrementally(input_directory, output_directory):
    # Ensure the output directory exists
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    group_name = os.path.basename(input_directory)
    output_filename = os.path.join(output_directory, f"{group_name}_lis_all_ranks.csv")

    # Check if a combined file already exists in the output directory
    existing_data = None
    if os.path.exists(output_filename):
        existing_data = pd.read_csv(output_filename)  # defaults to comma as separator
        # Check if 'source_file' column exists, if not, create it
        if 'source_file' not in existing_data.columns:
            existing_data['source_file'] = pd.NA  # Initialize with missing values
        existing_files = set(existing_data['source_file'].dropna().unique())
    else:
        existing_files = set()

    # Dataframe to store new data
    new_data = []

    # Iterate over all tsv files in the input directory
    for filename in os.listdir(input_directory):
        if filename.endswith('.tsv') and filename not in existing_files:
            # Assuming the original TSV files are also separated by tabs
            try:
                df = pd.read_csv(os.path.join(input_directory, filename), sep='\t')
                df['source_file'] = filename  # Add a column to track the source file
                new_data.append(df)
            except pd.errors.EmptyDataError:
                print(f"Skipping empty file: {filename}")

    # Combine new data with existing data if there is any new data
    if new_data:
        new_combined_df = pd.concat(new_data, ignore_index=True)
        if existing_data is not None:
            new_combined_df = pd.concat([existing_data, new_combined_df], ignore_index=True)
        # Save the updated combined data to the output directory without specifying sep
        new_combined_df.to_csv(output_filename, index=False)
    else:
        print(f"No new data to combine in {input_directory}")


# ColabFold LIS Analysis Script
This analyzes ColabFold outputs stored in {base_path} in the server.

It processes multiple result folders, computes LIS metrics, and aggregates results.


### Expected Directory Structure:
    colabfold_output/
    ├── folder_1/  (Contains .pdb and .json prediction results)
    ├── folder_2/
    ├── ...

### Output:
- LIS analysis results saved in {output_base} folder.



In [ ]:
import os
from datetime import datetime

def get_subdirectories(base_path):
    subdirectories = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
    subdirectories_with_times = [(d, os.path.getmtime(os.path.join(base_path, d))) for d in subdirectories]
    
    # Debug: Print the directories with their raw and local time modification times
#     print("Directories with their modification times:")
#     for d, t in subdirectories_with_times:
#         print(f"Directory: {d}, Last Modified Raw: {t}, Last Modified Local: {datetime.fromtimestamp(t)}")
    
    # Sort subdirectories by modification time in descending order
    sorted_subdirectories = sorted(subdirectories_with_times, key=lambda x: x[1], reverse=True)
    
    # Debug: Print the sorted directories
#     print("Sorted directories by modification time:")
#     for d, t in sorted_subdirectories:
#         print(f"Directory: {d}, Last Modified: {datetime.fromtimestamp(t)}")
    
    # Ensure sorting is correct
    assert all(sorted_subdirectories[i][1] >= sorted_subdirectories[i+1][1] for i in range(len(sorted_subdirectories) - 1)), "Sorting failed!"

    return [d[0] for d in sorted_subdirectories]


# Parameters
pae_cutoff = 12
distance_cutoff=8

base_path = "colabfold_output" # change this
output_base = f"analysis_pae_{pae_cutoff}" 
saving_all_rank_folder = f"{output_base}_all_ranks"
contact_lis_folder = output_base

# Generate folders_to_analyze list
folders_to_analyze = get_subdirectories(base_path)

# Main processing loop
for iteration in range(1):  
    current_iteration_folders = set()

    for folder in folders_to_analyze:
        if folder not in current_iteration_folders:
            print(f"Processing {folder}")
            process_files_in_folder(folder, base_path, output_base, pae_cutoff=pae_cutoff, distance_cutoff=distance_cutoff) 
            print(f"Finished processing and combining results for {folder}")
            data_folder = os.path.join(contact_lis_folder, folder)
            combine_tsv_files_incrementally(data_folder, saving_all_rank_folder)
            current_iteration_folders.add(folder)

    print(f"Iteration {iteration + 1} complete.")



## LIS Analysis File Search & Processing Script

The searches for LIS analysis files in a given directory, 
matches them against specific search terms, and processes them for further analysis.

### Expected Directory Structure:
    analysis_pae_12_all_ranks/
    ├── screen_1_lis_all_ranks.csv
    ├── screen_2_lis_all_ranks.csv
    ├── screen_3_lis_all_ranks.csv


In [ ]:
import numpy as np
import os
import fnmatch
import glob
import pandas as pd
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

def find_files(directory, search_terms):
    # Initialize an empty list to store the matching files
    file_names = []
    # Iterate over all search terms
    for term in search_terms:
        # Use glob to find all files that contain the search term
        matching_files = glob.glob(f'{directory}/*{term}*.csv', recursive=True)
        for file in matching_files:
            file_name = os.path.basename(file)
            file_names.append(file_name)
    return file_names

threshold_file = "threshold_file" # change this

# Define the directory where the search starts
directory = 'analysis_pae_12_all_ranks' # change this

search_terms = [
"screen",
]

searched_files = find_files(directory, search_terms)
print(searched_files)

## LIS Analysis Data Processing

This processes LIS analysis files by:

- Searching for relevant .csv files based on search terms  
- Renaming columns  
- Calculating integrated LIS (iLIS) using the formula:  

  $iLIS = \sqrt{LIS \times cLIS}$  

- Extracting and applying thresholds from an external Excel threshold file  
- Calculating a score based on the defined threshold preferences  



In [ ]:
dfs = []
for searched_file in searched_files:
    df = pd.read_csv(f'{directory}/{searched_file}')
    dfs.append(df)  

df_combined = pd.concat(dfs, ignore_index=True)

column_mapping = {
    'Protein_1': 'Protein_1',
    'Protein_2': 'Protein_2',
    'Rank': 'Rank',
    'integrated Local Interaction Score (Interface)': 'iLIS',
    'Local Interaction Score (AB)': 'LIS_AB',
    'Local Interaction Score (BA)': 'LIS_BA',
    'Local Interaction Score (Interface)': 'LIS',
    'Local Interaction Area (AB)': 'LIA_AB',
    'Local Interaction Area (BA)': 'LIA_BA',
    'Local Interaction Area (Interface)': 'LIA',
    'Local Interaction Residue (A)': 'LIR_A',
    'Local Interaction Residue (B)': 'LIR_B',
    'Local Interaction Residue (Sum)': 'LIR',
    'Local Interaction pLDDT (A)': 'LIpLDDT_A',
    'Local Interaction pLDDT (B)': 'LIpLDDT_B',
    'Local Interaction pLDDT (Interface)': 'LIpLDDT',
    'Contact Local Interaction Score (AB)': 'cLIS_AB',
    'Contact Local Interaction Score (BA)': 'cLIS_BA',
    'Contact Local Interaction Score (Interface)': 'cLIS',
    'Contact Local Interaction Area (AB)': 'cLIA_AB',
    'Contact Local Interaction Area (BA)': 'cLIA_BA',
    'Contact Local Interaction Area (Interface)': 'cLIA',
    'Contact Local Interaction Residue (A)': 'cLIR_A',
    'Contact Local Interaction Residue (B)': 'cLIR_B',
    'Contact Local Interaction Residue (Sum)': 'cLIR',
    'Contact Local Interaction pLDDT (A)': 'cLIpLDDT_A',
    'Contact Local Interaction pLDDT (B)': 'cLIpLDDT_B',
    'Contact Local Interaction pLDDT (Interface)': 'cLIpLDDT',
    'Local Interaction Residue Indice A': 'LIR_indice_A',
    'Local Interaction Residue Indice B': 'LIR_indice_B',
    'Contact Local Interaction Residue Indice A': 'cLIR_indice_A',
    'Contact Local Interaction Residue Indice B': 'cLIR_indice_B',
    'pLDDT': 'pLDDT',
    'pTM': 'pTM',
    'ipTM': 'ipTM',
    'Protein A Length': 'Protein_Len_A',
    'Protein B Length': 'Protein_Len_B',
    'pdb_file': 'pdb_file',
    'pae_json': 'pae_json',
    'pae_plot': 'pae_plot',
    'confidence': 'Confidence'
}

df_combined = df_combined.rename(columns=column_mapping)

df_combined['folder'] = df_combined['pae_plot'].str.split('+').str[0]
df_combined['Total length'] = df_combined['Protein_Len_A'] + df_combined['Protein_Len_B']

# Assuming df_final is your DataFrame and already has 'LIS' and 'cLIS'
df_combined['LIS*cLIS'] = df_combined["LIS"] * df_combined["cLIS"]

# Apply min-max scaling using constants
df_combined['iLIS'] = np.sqrt(df_combined['LIS*cLIS'])
df_combined = df_combined.drop(columns=['LIS*cLIS'])

df_threshold_yfh = pd.read_excel(threshold_file)

# # Ensure the gene IDs and proteins are treated as strings
df_combined['Protein_1'] = df_combined['Protein_1'].astype(str)
df_combined['Protein_2'] = df_combined['Protein_2'].astype(str)

# Extract thresholds for each metric
metrics = ['iLIS', 'LIS', 'cLIS', 'ipTM', 'Confidence'] 
thresholds = {}
for metric in metrics:
    metric_thresholds = df_threshold_yfh[(df_threshold_yfh['Group'] == 'Total') & (df_threshold_yfh['Column'] == metric)]
    if not metric_thresholds.empty:
        metric_thresholds = metric_thresholds.iloc[0]
        thresholds[metric] = {
            'Threshold@0.10': metric_thresholds['Threshold@0.10'],
            'Threshold@0.05': metric_thresholds['Threshold@0.05'],
            'Threshold@0.01': metric_thresholds['Threshold@0.01']
        }
    else:
        print(f"Thresholds for metric '{metric}' not found. Please check the input data.")

# Change score depending on preference
def calculate_score(row, thresholds):
    score = 0
    for metric in thresholds:
        value = row[metric]
        if value >= thresholds[metric]['Threshold@0.01']:
            score += 1
        elif value >= thresholds[metric]['Threshold@0.05']:
            score += 1
        elif value >= thresholds[metric]['Threshold@0.10']:
            score += 1
    return score


# Apply the score calculation function to each row
df_combined['Score'] = df_combined.apply(lambda row: calculate_score(row, thresholds), axis=1)
df_combined = df_combined[~df_combined.duplicated(subset=['Rank', 'pae_plot'])]
 
# Save the results to an Excel file
df_plot = df_combined[['Protein_1', 
             'Protein_2',
             'Rank',
             'Score',
             'iLIS', 
             'LIS', 
             'cLIS', 
             'ipTM', 
             'Confidence',
             'pae_plot',
             ]] 

df_plot.to_csv(f'{search_terms[0]}_summary.csv', index=False)
print(f'{search_terms[0]}_summary.csv has been saved.')



## Summary and Full Data  

- **Summary** can be found in the CSV file generated from the above code.  
- **Full detailed information** is available in the CSV file generated from the code below.  








In [ ]:
import re

def extract_numbers(s):
    if isinstance(s, str):
        numbers = re.findall(r'\d+', s)
        return ', '.join(numbers)
    return s

columns_to_clean = ['LIR_indice_A', 'LIR_indice_B', 'cLIR_indice_A', 'cLIR_indice_B']

for col in columns_to_clean:
    df_combined[col] = df_combined[col].apply(extract_numbers)

def process_list_column(value):
    if isinstance(value, str):
        combined_set = set()
        # Replace '[' and ']' with '', then split by commas and convert to integers
        value = value.replace('[', '').replace(']', '')
        for part in value.split(','):
            part = part.strip()
            if part and part != '...':
                combined_set.update(map(int, part.split()))
        return ','.join(map(str, sorted(combined_set)))
    return value


df_new = df_combined.copy()

columns_to_clean = ['LIR_indice_A', 'LIR_indice_B', 'cLIR_indice_A', 'cLIR_indice_B']

# Define the columns that contain lists
list_columns = ['LIR_indice_A', 'LIR_indice_B', 'cLIR_indice_A', 'cLIR_indice_B']

for col in columns_to_clean:
    df_new[col] = df_new[col].apply(lambda x: re.sub(r'\[\s+', '[', x) if isinstance(x, str) else x)    
    
# Post-process list columns to combine all unique coordinates
for col in list_columns:
    df_new[col] = df_new[col].apply(process_list_column)

import pandas as pd

# Define the original mapping (short -> full name)
column_mapping_reverse = {v: k for k, v in column_mapping.items()}

# Define README column mapping section
readme_column_data = pd.DataFrame([
    {'Column Name': short, 'Full Description': full}
    for short, full in column_mapping_reverse.items()
])

# Define the yeast-fly-human reference PPI benchmark metrics table
benchmark_metrics_data = pd.DataFrame([
    ['Total', 'iLIS', 0.22252179, 0.735465116, 0.101574803, 0.338551792, 0.688953488, 0.052755906, 0.550663967, 0.406976744, 0.011023622, 0.291922729, 0.720930233, 0.063779528],
    ['Total', 'LIS', 0.168237963, 0.75872093, 0.101574803, 0.256569551, 0.686046512, 0.051968504, 0.438797151, 0.39244186, 0.01023622, 0.203428117, 0.73255814, 0.070866142],
    ['Total', 'cLIS', 0.297521879, 0.735465116, 0.103149606, 0.4640458, 0.680232558, 0.050393701, 0.715617835, 0.39244186, 0.01023622, 0.21830786, 0.787790698, 0.134645669],
    ['Total', 'ipTM', 0.48, 0.639534884, 0.105511811, 0.59, 0.520348837, 0.051968504, 0.72, 0.36627907, 0.011023622, 0.43, 0.683139535, 0.133070866],
    ['Total', 'Confidence', 0.494, 0.610465116, 0.1, 0.576, 0.511627907, 0.050393701, 0.684, 0.363372093, 0.01023622, 0.41, 0.700581395, 0.172440945]
], columns=[
    'Group', 'Metric', 'Threshold@0.10', 'TPR@0.10', 'FPR@0.10', 
    'Threshold@0.05', 'TPR@0.05', 'FPR@0.05', 'Threshold@0.01', 
    'TPR@0.01', 'FPR@0.01', 'Youden_Threshold', 'Youden_TPR', 'Youden_FPR'
])

# Reordering the columns
desired_order = ['Protein_1', 'Protein_2', 'Rank', 'iLIS', 'LIS', 'cLIS', 'ipTM', 'Confidence', 'pLDDT', 'pTM']
remaining_columns = [col for col in df_new.columns if col not in desired_order]
new_column_order = desired_order + remaining_columns

# Reorder the dataframe
df_new = df_new[new_column_order]

# Define iLIS explanation with only the bold title
ilis_explanation = pd.DataFrame([
    ["iLIS = sqrt(LIS * cLIS)"],
    [""]
], columns=["Description"])

# Save to Excel and apply formatting
with pd.ExcelWriter(f'{search_terms[0]}_combined.xlsx', engine='xlsxwriter') as writer:
    df_new.to_excel(writer, sheet_name='Data', index=False)

    # Access the workbook and Data worksheet
    workbook = writer.book
    data_worksheet = writer.sheets['Data']
    
    # Apply wrapped text format for column headers
    wrap_format = workbook.add_format({'text_wrap': True, 'bold': True, 'align': 'center', 'valign': 'vcenter'})

    # Set column widths and apply wrap format
    for col_num, value in enumerate(df_new.columns):
        # data_worksheet.write(0, col_num, value, wrap_format)
        data_worksheet.set_column(col_num, col_num, 9)
    
    # Write README sheet
    workbook = writer.book
    worksheet = workbook.add_worksheet('README')
    writer.sheets['README'] = worksheet

    # Define bold format
    bold_format = workbook.add_format({'bold': True})

    # Write bold title
    worksheet.write(0, 0, "integrated LIS (iLIS) Calculation", bold_format)

    # Write formula below title
    worksheet.write(1, 0, "iLIS = sqrt(LIS * cLIS)")

    # Write column mapping below explanation
    readme_column_data.to_excel(writer, sheet_name='README', index=False, startrow=3)

    # Write performance metrics below column mapping
    benchmark_metrics_data.to_excel(writer, sheet_name='README', index=False, startrow=len(readme_column_data) + 6)

print(f'{search_terms[0]}_combined.xlsx with README containing benchmark results for each metric created.')

